# Grappling with a CNN

This notebook will convince me that I can build a CNN.

In [1]:
# Libraries
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import tensorflow.keras as keras
import tensorboard
import psycopg2
from PIL import Image
import os
import h5py
import datetime
%load_ext tensorboard
!rm -rf ./logs/

The data are stored locally in a PostgreSQL database. So let's connect and get them.

In [13]:
# Read in data from origami database in PostgreSQL
try:
    print('Attempting to connect to PostgreSQL.')
    conn = None
    conn = psycopg2.connect(host='localhost', database='origami', user='postgres', password='postgres')
    cur = conn.cursor()
    # SELECT image classifications and file paths from PostgreSQL
    sql_select = "SELECT image_class, image_path FROM origami_images;"
    cur.execute(sql_select)
    print('Selecting rows from origami_images table.')
    origami_images = cur.fetchall()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        cur.close()
        conn.close()
        print('PostgreSQL connection closed.')

Attempting to connect to PostgreSQL.
Selecting rows from origami_images table.
PostgreSQL connection closed.


This gives us `origami_images`, a list of tuples, each of which gives a class and a file path for the associated image.

In [14]:
print(len(origami_images))
print(origami_images[0])

1150
('butterfly', '/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/butterfly/63.jpg')


Let's shuffle this list randomly.

In [15]:
random.shuffle(origami_images)

We'll turn this into two `numpy` arrays to train our model. The class names are easy enough.

In [16]:
image_classes = []
class_names = []
for img in origami_images:
    image_classes.append(img[0])
    if img[0] not in class_names:
        class_names.append(img[0])
image_classes = np.array(image_classes)
class_names = np.array(class_names)
print(class_names)
print(image_classes.shape)

['frog' 'duck' 'crane' 'lotus' 'butterfly' 'star']
(1150,)


In [17]:
def get_label(label):
    return label == class_names

origami_y = []
for label in image_classes:
    origami_y.append(get_label(label))
origami_y = tf.convert_to_tensor(origami_y)
print(origami_y.shape)

(1150, 6)


The images are a little harder. Let's define a couple of functions to help with image processing.

In [18]:
# Side length of normalized image
IMG_SIZE = 256

# Function to decode an image, render in grayscale and square dimensions
def decode_img(img):
    img = tf.io.decode_image(img, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float64)
    img = tf.image.rgb_to_grayscale(img)
    return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

# Function to process a file path and return the image
def process_path(img_path):
    # Decode image using auxiliary function
    img = tf.io.read_file(img_path)
    img = decode_img(img)
    return img

OK, so now let's create our second array.

In [19]:
origami_X = []
counter = 0
for img in origami_images:
    print(img[1])
    origami_X.append(process_path(img[1]))
    counter += 1
    if counter % 25 == 0:
        print('{} files processed.'.format(counter))
print('All files processed. Converting to tensor.')
origami_X = tf.convert_to_tensor(origami_X)
print(origami_X.shape)

/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/frog/41.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/duck/118.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/106.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/98.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/88.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/frog/91.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/31.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/149.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/frog/17.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/115.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/duck/109.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/78.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/28.jpg

/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/frog/147.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/72.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/lotus/5.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/7.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/star/91.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/64.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/120.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/frog/152.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/star/181.jpg
/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/crane/117.png


InvalidArgumentError: Matrix size-incompatible: In[0]: [367500,4], In[1]: [3,1] [Op:MatMul] name: rgb_to_grayscale/Tensordot/MatMul/

OK, let's give this CNN thing a whirl...

In [21]:
cnn = keras.Sequential()
# Layer construction w/ L2 regularizers on convolutional layers
cnn.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
cnn.add(keras.layers.MaxPooling2D((2, 2)))
cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(16, activation='relu'))
cnn.add(keras.layers.Dense(len(class_names), activation='softmax'))
cnn.add(keras.layers.Reshape((-1,)))

Compile it.

In [22]:
cnn.compile(optimizer='adam',
           loss=keras.losses.CategoricalCrossentropy(from_logits=True),
           metrics=[keras.losses.CategoricalCrossentropy(from_logits=True),
                    'categorical_accuracy'])
cnn.summary()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1000000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                16000016  
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 85        
_________________________________________________________________
reshape_1 (Reshape)          (None, None)             

And fit it.

In [23]:
history = cnn.fit(origami_X, origami_y, epochs=16, callbacks=[tensorboard_callback])

Train on 166 samples
Epoch 1/16
166/166 [==============================] - 37s 222ms/sample - loss: 1.7031 - categorical_crossentropy: 1.6853 - categorical_accuracy: 0.1807
Epoch 2/16
166/166 [==============================] - 20s 121ms/sample - loss: 1.7060 - categorical_crossentropy: 1.6878 - categorical_accuracy: 0.1988
Epoch 3/16
166/166 [==============================] - 19s 117ms/sample - loss: 1.7060 - categorical_crossentropy: 1.7330 - categorical_accuracy: 0.1988
Epoch 4/16
166/166 [==============================] - 18s 109ms/sample - loss: 1.7060 - categorical_crossentropy: 1.6878 - categorical_accuracy: 0.1988
Epoch 5/16
166/166 [==============================] - 19s 117ms/sample - loss: 1.7060 - categorical_crossentropy: 1.6652 - categorical_accuracy: 0.1988
Epoch 6/16
166/166 [==============================] - 18s 110ms/sample - loss: 1.7060 - categorical_crossentropy: 1.7104 - categorical_accuracy: 0.1988
Epoch 7/16
166/166 [==============================] - 17s 104ms/sam

In [24]:
%tensorboard --logdir logs/fit